In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Back to fully parenthesised expressions

In [135]:
from collections import defaultdict
import sys

class p_tree:
    parentheses = {')':'(',']':'[','}':'{'}
    operators = {'+':lambda x,y:x+y,'-':lambda x,y:x-y,'*':lambda x,y: x*y,'/':lambda x,y: x/y}
    tree = None
    root = None
    def __init__(self, expression = None):
        self.expression = expression
    #assume all expressions inputted are legal
    def transform(self):
        new_exp = []
        nb = ''
        for s in self.expression:
            if s != ' ':
                if s in self.parentheses or s in self.operators or s in self.parentheses.values():
                    if nb != '':
                        new_exp.append(int(nb))
                        nb = ''
                    new_exp.append(s)
                else:
                    nb += s
        if new_exp == [] and nb != '':
            new_exp.append(int(nb))
        return new_exp
    def evaluate(self):
        #using  list as stack
        new_exp = self.transform()
        if len(new_exp) == 1:
    #         if int(new_exp[0]) == new_exp[0]:
            if isinstance(new_exp[0],int):
            #Return whether an object is an instance of a class or of a subclass thereof
                return new_exp[0]
        stack = []
        result = None
        operator = None
        for s in new_exp:
            if s in self.parentheses:
                result = stack.pop()
                if self.parentheses[s] != stack.pop():
                    return
                while stack:
                    t = stack.pop()
                    if t in self.operators:
                        result = self.operators[t](stack.pop(),result)
                    else:
                        stack.append(t)
                        break
                stack.append(result)
            else:
                if s in self.operators or s in self.parentheses.values():
                    stack.append(s)
                else:
                    if stack == []:
                        stack.append(s)
                    temp = stack.pop()
                    if temp in self.operators:
                        result = stack.pop()
                        result = self.operators[temp](result,s)
                        while stack:
                            t1 = stack.pop()
                            if t1 in self.operators:
                                result = self.operators[t1](stack.pop(),result)
                            else:
                                stack.append(t1)
                                break
                        stack.append(result)
                    else:
                        stack.append(temp)
                        stack.append(s)
        return result
    def build_tree(self):
        new_exp = self.transform()
        ## Algorithm to build the tree of arithmetic expressions:
        #step 0: if {'(','[','{'}, then append into stack
        #step 1: if is a number, pop stack:
            # if is {'(','[','{'}, then append back and append the number into stack
            # if is operator, the number is a child of the operator
        #step 2: if operator, pop stack:
            # if is a number, then as one of child of this operator
            # if is another operator, then this popped operator as one of child of the previous operator
            # if {'(','[','{'} , append back
            # append the operator
        #step 3: if oppsite of {'(','[','{'} , pop stack:
            # first pop must be an operator, record it
            # pop again, second pop must be one of {'(','[','{'}
            # third pop,
                # if one of {'(','[','{'}, append back and then append the operator
                # if is an operator, the previous operator is a child of this operator, append the latter popped operator back
        # start over
        tr = defaultdict(list)
        stack = []
        for s in new_exp:
            if s in self.parentheses.values():
                stack.append(s)
            if isinstance(s,int):
                temp = stack.pop()
                if temp in self.parentheses.values():
                    stack.append(temp)
                    stack.append(s)
                if temp in self.operators:
                    tr[temp].append(s)
                    stack.append(temp)
            if s in self.operators:
                temp = stack.pop()
                if isinstance(temp,int):
                    tr[s].append(temp)
                if temp in self.operators:
                    tr[s].append(temp)
                if temp in self.parentheses.values():
                    stack.append(temp)
                stack.append(s)
            if s in self.parentheses:
                # t1 must be an operator
                t1 = stack.pop()
                t2 = stack.pop()
                if not stack:
                    stack.append(t1)
                else:
                    t3 = stack.pop()
                    if t3 in self.parentheses.values():
                        stack.append(t3)
                        stack.append(t1)
                    if t3 in self.operators:
                        tr[t3].append(t1)
                        stack.append(t3)
        self.tree = tr
    def find_root(self):
        new_exp = self.transform()
        # find the root operator:
            # look up in the expression, count the parentheses
            # if left parentheses, then count + 1, otherwise count - 1
            # if meet an operator, then record in a dictionary(operator and index to be the key, count to be the value)
        # after the loop ends, find the operator with count being 1
        counter = 0
        find_root = {}
        for i in range(len(new_exp)):
            if new_exp[i] in self.parentheses.values():
                counter += 1
            if new_exp[i] in self.parentheses:
                counter -= 1
            if new_exp[i] in self.operators:
                find_root[(new_exp[i],i)] = counter
        for ro, index in find_root:
            if find_root[(ro,index)] == 1:
                self.root = ro
    def height(self):
        if self.root is None:
            return 0
        else:
            return max(self._height(child) for child in self.tree[self.root])
    def _height(self,node):
        if node is None:
            return 0
        elif node not in self.tree:
            return 1
        else:
            return max(self._height(child) for child in self.tree[node]) + 1           
    def print_binary_tree(self):
        self.build_tree()
        self.find_root()
        def display(node,level,height):
            if node not in self.tree:
                print('\n'*(2**(height - level)-1))
                print('     '*level,node,sep='')
            else:
                display(self.tree[node][0],level+1,height)
                print('     '*level,node,sep='')
#                 print('\n'*(2**level + 2**(height - level - 1)-1),'  '*(level+3)+str(self.tree[node][1]), sep='')
                display(self.tree[node][1],level+1,height)
        display(self.root,0,self.height())

In [136]:
parse_tree = p_tree('[(1-20)+300]')
parse_tree.print_binary_tree()
parse_tree.height()


          1
     -

          20
+


     300


2

In [52]:
sum(x for x in [1,2,3,4,45])

55

In [ ]:
def height(self):
    if self.root is None:
        return 0
    else:
        return max(self._height(child) for child in self.tree[self.root])
#         return max(self._height(self.tree[self.root][0]),self._height(self.tree[self.root][1])) + 1
def _height(self,node):
    if node not in self.tree:
        return 1
    else:
        return max(self._height(child) for child in self.tree[self.root]) + 1
#         return max(self._height(self.tree[node][0]),self._height(self.tree[node][1]))+1
def display(node,level,height):
    if node not in self.tree:
        print('\n'*(2**(height - level)-1),sep='')
        print('    '*level, node, sep='')
    else:
        display(self.tree[node][0],level+1,height)
        print('    '*level,node,sep = '')
        display(self.tree[node][1],level+1,height)
        
## Eric's code---practise
# 1:
def height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left._height(),self.right._height())
def _height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left.height(),self.right._height()) + 1
def display(self):
    self._display(0,self.height)
def _display(self,level,height):
    if level > height:
        return
    if self.value is None:
        print('\n'*(2**(height - level+1)-1))
    else:
        self.left._display(level+1,height)
        print('    '*level,self.value,sep='')
        self.right._display(level+1,height)
# 2:
def height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left._height(),self.right._height())
def _height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left._height(),self.right._height())+1
def display(self):
    self._display(0,self.height)
def _display(self,level,height):
    if level > height:
        return
    if self.value is None:
        print('\n'*(2**(height-level+1)-1))
    else:
        self.left._display(level+1,height)
        print('   '*level,self.value,sep = '')
        self.right._display(level+1,height)
# 3:
def height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left._height(),self.right._height())
def _height(self):
    if self.value is None:
        return 0
    else:
        return max(self.left._height(),self.right._height()) + 1
def display(self):
    if level > height :
        return
    if self.value is None:
        print('\n'*(2**(height - level +1)-1))
    else:
        self.left._display(level+1,height)
        print('   '*level,self.value,sep='')
        self.right._display(level+1,height)

In [55]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.

